In [1]:
import sys
from collections import Counter

import dateutil.parser
import re
import datetime as dt

import numpy as np
import sklearn.cross_validation as cvl
import sklearn.feature_extraction.text as txt
import nltk
import requests
from bs4 import BeautifulSoup

In [2]:
import psycopg2 as db
conn = db.connect("dbname=TBM")
cur = conn.cursor()

In [3]:
def preprocess(title):
    title = title.replace('U.S.','USA')
    title = title.replace('.com',' __dotcom__')
    title = title.replace("'s",'')
    title = title.replace("\'",' ')
    title = title.replace('\"',' ')
    title = title.replace('-',' ')
    title = title.replace('.',' ')
    title = title.replace(',',' ')
    title = title.replace('(',' ')
    title = title.replace(')',' ')
    title = title.replace('/',' ')
    title = title.replace(';',' ')
    title = title.replace(':',' ')
    title = title.replace(' to ',' ')
    title = title.replace(' in ',' ')
    title = title.replace(' for ',' ')
    title = title.replace(' on ',' ')
    title = title.replace(' as ',' ')
    title = title.replace(' of ',' ')
    title = title.replace(' by ',' ')
    title = title.replace(' at ',' ')
    title = title.replace('$',' $ ')
    title = title.replace('UPDATE',' ')
        
    return [w.lower() for w in title.split() if w is '$' or len(w) >= 2 and w.isalpha()]

In [4]:
class Titles:
    def __init__(self,n='ALL'):
        if isinstance(n,int):
            n = str(n)
        select_query = 'SELECT feedly_id,title FROM reuters_usmarkets LIMIT %s' % n
        self.batch_size = 1000
        cur.execute(select_query)
        self.records = []
        
    def __iter__(self):
        return self

    def __next__(self):
        if not self.records:
            self.records = cur.fetchmany(self.batch_size)
            if not self.records:
                raise StopIteration()
        next_article = self.records.pop(0)
        return preprocess(next_article[1])

titles = list(Titles())

In [5]:
c = Counter()
for title in titles:
    c.update(title)

In [6]:
c.most_common(20)

[('usa', 24948),
 ('$', 15366),
 ('stocks', 14933),
 ('says', 13902),
 ('after', 9657),
 ('us', 9174),
 ('new', 8251),
 ('with', 8146),
 ('canada', 7792),
 ('fed', 7678),
 ('shares', 7564),
 ('up', 7524),
 ('china', 7515),
 ('markets', 7510),
 ('pct', 7279),
 ('bln', 6782),
 ('data', 6590),
 ('global', 6563),
 ('bank', 6421),
 ('brief', 6287)]

In [7]:
def title_score(title):
    return np.log(sum([c[w] for w in title])+1)

In [9]:
test = titles[:1000]
top10 = np.argsort([title_score(title) for title in test])[-50:]
[' '.join(test[i]) for i in top10]

['usa treasury sell $ bln bills',
 'doubleline gundlach says usa equity markets face another major leg down',
 'wall st slide unlikely undermine support usa stocks',
 'global markets grim china data caps worst week year stocks',
 'rpt global markets stocks oil tumble grim china pmi sparks growth fears',
 'corrected cbo says usa revenue gains push down deficit delay default',
 'firm that vetted snowden $ million settlement with usa',
 'usa stock futures tumble month low after weak china pmi',
 'usa treasury sells $ bln tips record investor demand',
 'usa treasury sells $ bln tips record investor demand',
 'moves usa bank hires susie roh from guaranty bank and trust',
 'usa inflation probably lower than reported fed study says',
 'us stocks snapshot futures up sharply day after global rout',
 'oshkosh wins $ bln deal replace usa army marine humvees',
 'usa treasury sell $ bln floating rate notes',
 'us stocks wall st set fall weak chinese data adds growth fears',
 'global bond yields eur

In [15]:
def get_top_articles(url):
    # url like http://www.reuters.com/resources/archive/us/20121223.html
    r = requests.get(url)
    soup = BeautifulSoup(r.text,'lxml')
    soup = soup.find('div',class_='primaryContent')
    soup = soup.find('div',class_='module')
    articles = soup.find_all('div',class_='headlineMed')

    links = []; titles = []; 
    for article in articles:
        a_tag = article.find('a')
        link = a_tag.get('href')
        title = a_tag.text
        links.append(link)
        titles.append(title)
    
    n_articles = min(len(titles),100)
    scores = [title_score(preprocess(title)) for title in titles]
    top_idx = np.argsort(scores)[-n_articles:]
    titles = [titles[i] for i in top_idx]
    links = [links[i] for i in top_idx]

    return links,titles

In [16]:
def get_contents(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    date = soup.find('span',class_='timestamp').text
    date = dateutil.parser.parse(date)
    title = soup.find('h1',class_='article-headline').text
    content = soup.find(id='articleText').text

    return date,content

In [17]:
def get_articles_by_date(date):
    url = 'http://www.reuters.com/resources/archive/us/%s.html' % date.strftime('%Y%m%d')
    links,titles = get_top_articles(url)
    
    dates = []; contents = []
    for url in links:
        try:
            date,content = get_contents(url)
            content = content.strip()
            dates.append(date)
            contents.append(content)
        except Exception as e:
            print(e,url)
            dates.append(None)
            contents.append(None)

    links = [link for link,date in zip(links,dates) if date is not None]
    titles = [title for title,date in zip(titles,dates) if date is not None]
    dates = [date for date in dates if date is not None]
    contents = [content for content in contents if content is not None]

    return dates,titles,contents,links

In [18]:
def insert_in_db(dates,titles,contents,links):
    query = 'INSERT INTO articles3 (date,title,content,link) VALUES (%s,%s,%s,%s)'

    for val in zip(dates,titles,contents,links):
        cur.execute(query,val)

    conn.commit()

In [19]:
date = dt.date(2014,10,29)
end = dt.date(2013,12,31)
day = dt.timedelta(days=1)

while date != end:
    print(date)
    url = 'http://www.reuters.com/resources/archive/us/%s.html' % date.strftime('%Y%m%d')
    dates,titles,contents,links = get_articles_by_date(date)
    insert_in_db(dates,titles,contents,links)
    date = date - day

 http://www.reuters.com/article/2014/01/01/mrclean-timessquare-idUSnPnMMfFPnh+16c+PRN20140101


 http://www.reuters.com/article/2014/01/02/mi-uaw-retiree-trust-idUSnPnDEgztJ6+162+PRN20140102
'NoneType' object has no attribute 'text'

 http://www.reuters.com/article/2014/01/01/bc-ac-idUSnPnjsB8hn+88+PRN20140101
'NoneType' object has no attribute 'text'

 http://www.reuters.com/article/2014/01/01/farmers-thankamillion-idUSnPnMMgjSp8+164+PRN20140101
'NoneType' object has no attribute 'text'

 http://www.reuters.com/article/2014/01/01/toshiba-vision-ny-idUSnPnHKgyW9C+165+PRN20140101
'NoneType' object has no attribute 'text'

...

 http://www.reuters.com/article/2014/10/28/compuverde-uspatent-idUSnPn7MRF3+4e+PRN20141028
'NoneType' object has no attribute 'text'

 http://www.reuters.com/article/2014/10/28/lionsgate-conf-call-idUSnPn4GMyp9+98+PRN20141028
'NoneType' object has no attribute 'text'

 http://www.reuters.com/article/2014/10/28/de-dupont-earnings-idUSnPn7dRs3d+88+PRN20141028
'NoneType' object has no attribute 'text'

 http://www.reuters.com/article/2014/10/28/ca-plastc-record-idUSnPn2zXLRZ+8f+PRN20141028
'NoneType' object has no attribute 'text'

2014-10-29
'NoneType' object has no attribute 'text'

In [20]:
2+3

5